In [1]:
import pandas as pd

df= pd.read_csv('data/test/finaltest.csv')
df2= pd.read_csv('data/test/multi_log_reg/multi_ftse.csv')

In [3]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Ticker,date,change,num,sum,max,avg,min_dist,max_dist
0,0,2,2,^FTSE,2008-08-01 00:00:00+00:00,-57.199707,0,0.0,0.0,0.00,0.000000,0.000000
1,1,14,14,^FTSE,2008-08-04 00:00:00+00:00,-34.500000,0,0.0,0.0,0.00,0.000000,0.000000
2,2,26,26,^FTSE,2008-08-05 00:00:00+00:00,134.299805,2,12.3,6.3,6.15,8201.703622,13210.719473
3,3,38,38,^FTSE,2008-08-06 00:00:00+00:00,31.600098,0,0.0,0.0,0.00,0.000000,0.000000
4,4,50,50,^FTSE,2008-08-07 00:00:00+00:00,-8.600098,0,0.0,0.0,0.00,0.000000,0.000000


In [5]:
df2 = df2.rename(columns={'Date': 'date'})
df2['date'] = pd.to_datetime(df2['date']).dt.tz_localize('UCT')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%dT%H:%M:%S%z')
df2['date'] = pd.to_datetime(df2['date']).dt.strftime('%Y-%m-%dT%H:%M:%S%z')
df2_subset = df2[['date', 'effect']]

# Merge df and df2_subset on the Date column
merged_df = pd.merge(df, df2_subset, on='date', how='left')
merged_df.head()


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Ticker,date,change,num,sum,max,avg,min_dist,max_dist,effect
0,0,2,2,^FTSE,2008-08-01T00:00:00+0000,-57.199707,0,0.0,0.0,0.00,0.000000,0.000000,0
1,1,14,14,^FTSE,2008-08-04T00:00:00+0000,-34.500000,0,0.0,0.0,0.00,0.000000,0.000000,0
2,2,26,26,^FTSE,2008-08-05T00:00:00+0000,134.299805,2,12.3,6.3,6.15,8201.703622,13210.719473,0
3,3,38,38,^FTSE,2008-08-06T00:00:00+0000,31.600098,0,0.0,0.0,0.00,0.000000,0.000000,0
4,4,50,50,^FTSE,2008-08-07T00:00:00+0000,-8.600098,0,0.0,0.0,0.00,0.000000,0.000000,0


In [6]:
merged_df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1','Unnamed: 0'], inplace=True)
merged_df.head()

,Ticker,date,change,num,sum,max,avg,min_dist,max_dist,effect
0,^FTSE,2008-08-01T00:00:00+0000,-57.199707,0,0.0,0.0,0.00,0.000000,0.000000,0
1,^FTSE,2008-08-04T00:00:00+0000,-34.500000,0,0.0,0.0,0.00,0.000000,0.000000,0
2,^FTSE,2008-08-05T00:00:00+0000,134.299805,2,12.3,6.3,6.15,8201.703622,13210.719473,0
3,^FTSE,2008-08-06T00:00:00+0000,31.600098,0,0.0,0.0,0.00,0.000000,0.000000,0
4,^FTSE,2008-08-07T00:00:00+0000,-8.600098,0,0.0,0.0,0.00,0.000000,0.000000,0


In [7]:
merged_df['effect'].unique()

array([0, 1, 3, 2])

In [9]:
import statsmodels.api as sm

# Add a constant to the independent variables (for the intercept term)
X = sm.add_constant(merged_df[['num', 'max', 'min_dist']])
y = merged_df['effect']

# Fit the logistic regression model
mlogit_model = sm.MNLogit(y, X)
result = mlogit_model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.720127
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:                 effect   No. Observations:                 3784
Model:                        MNLogit   Df Residuals:                     3772
Method:                           MLE   Df Model:                            9
Date:                Sat, 22 Feb 2025   Pseudo R-squ.:                0.001646
Time:                        11:59:57   Log-Likelihood:                -2725.0
converged:                       True   LL-Null:                       -2729.5
Covariance Type:            nonrobust   LLR p-value:                    0.4386
  effect=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.2794      0.050    -25.362      0.000      -1.378      -1.181
num           -0.0133      0.